In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from tensorflow.compat.v1.keras.backend import get_session
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

os.environ["CUDA_VISIBLE_DEVICES"]="0" # second gpu
gpus = tf.config.list_physical_devices("GPU")
gpu_to_limit = gpus[0]  # or 1, watch out for CUDA_VISIBLE_DEVICES
gpu_mem_mb = 24564  # tyle mamy
tf.config.set_logical_device_configuration(
    gpu_to_limit, [tf.config.LogicalDeviceConfiguration(memory_limit=gpu_mem_mb//2)]
)

2025-04-09 06:29:46.920751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 06:29:50.401675: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jovyan/.conda/envs/env-3.8/lib/:/home/jovyan/.conda/envs/env-3.8/lib/python3.8/site-packages/nvidia/cudnn/lib:/home/jovyan/.conda/envs/env-3.8/lib/:/home/jovyan/.conda/envs/env-3.8/lib/python3.8/site-packages/nvidia/cudnn/lib:
2025-04-09 06:29:50.401867: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import cycle
from ruptures.utils import pairwise
from sklearn.model_selection import train_test_split
import graphviz

from sklearn import tree
from sklearn.tree import  export_graphviz
from graphviz import Source
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from IPython.display import display
from IPython.display import SVG
import shap
import matplotlib.pyplot as plt
import ruptures as rpt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from sklearn.tree import DecisionTreeClassifier
from aeon.datasets import load_classification
from windowshap import SlidingWindowSHAP, StationaryWindowSHAP, DynamicWindowSHAP
import time
from tslearn.clustering import KShape
from kshape.core import KShapeClusteringCPU
from kshape.core_gpu import KShapeClusteringGPU
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.cluster import KMeans

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [6]:
import tsproto
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from scipy import stats 
import xgboost as xgb

# Create model architecture

In [7]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Reshape
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D,ConvLSTM1D
from keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
import math

def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor
 
def load_dataset_aeon(dsname):
    X, y, meta_data = load_classification(dsname)
    X = np.moveaxis(X,1,2)
    
    le = LabelEncoder()
    y = le.fit_transform(y)
    trainX, testX, trainy, testy= train_test_split(X,y)
    
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    return trainX, trainy, testX, testy

def generate_label_weights(train_y):
    # Count the occurrences of each class by summing across rows
    class_counts = np.sum(train_y, axis=0)

    # Calculate the total number of samples
    total_samples = np.sum(class_counts)

    # Calculate the weight for each class
    class_weights = {}
    num_classes = train_y.shape[1]
    for i in range(num_classes):
        class_weights[i] = total_samples / (num_classes * class_counts[i])

    return class_weights

def train_lstmconv_model(trainX, trainy, testX, testy, kernel=9):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)

    dg = divisorGenerator(n_timesteps)
    n_steps = int([next(dg) for i in range(3)][-1])
    n_length = int(n_timesteps/n_steps)
    # define model
    print(f'Input shape (time, rows, channels): {(n_steps, n_length, n_features)}')
    model = Sequential()
    model.add(Reshape((n_steps, n_length, n_features), input_shape=trainX.shape[1:]))
    model.add(ConvLSTM1D(filters=64, kernel_size=kernel, padding='same', activation='relu', strides=1, 
                         data_format='channels_last',
                         return_sequences=True,
                         input_shape=(n_steps, n_length, n_features)))
    model.add(ConvLSTM1D(filters=32, kernel_size=kernel, padding='same', activation='relu', strides=1, 
                         data_format='channels_last',
                         return_sequences=True,
                         input_shape=(n_steps, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten(name='embedding')) #is this a latent representation we could use for clustering?
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    

    print(model.summary())
    # fit network
    weights = generate_label_weights(trainy)
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,class_weight=weights)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy, model

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 

# run an experiment
def evaluate_lstmconv_model(dataset, repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset_aeon(dataset)

    #fill missing with zeros
    trainX = np.nan_to_num(trainX, nan=0)
    testX = np.nan_to_num(testX, nan=0)
    
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX = scaler.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score,model = train_lstmconv_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    return model, trainX, trainy, testX,testy,score
 

In [8]:
import pickle
import os
import keras
def save_bundle(model, trainX,trainy,testX,testy,svtr,svts, dsname, dir='./results'):
    if not os.path.isdir(f'{dir}/{dsname}'):
        os.makedirs(f'{dir}/{dsname}')

    model.save(f'{dir}/{dsname}/model.h5',save_format='h5')
    pickle.dump(trainX, open(f'{dir}/{dsname}/trainX.pickle','wb'))
    pickle.dump(trainy, open(f'{dir}/{dsname}/trainy.pickle','wb'))
    pickle.dump(testX, open(f'{dir}/{dsname}/testX.pickle','wb'))
    pickle.dump(testy, open(f'{dir}/{dsname}/testy.pickle','wb'))
    pickle.dump(svtr, open(f'{dir}/{dsname}/svtr.pickle','wb'))
    pickle.dump(svts, open(f'{dir}/{dsname}/svts.pickle','wb'))

    
def load_bundle(dsname, dir='./results'):
    model = keras.models.load_model(f'{dir}/{dsname}/model.h5')
    
    trainX= pickle.load( open(f'{dir}/{dsname}/trainX.pickle','rb'))
    trainy = pickle.load(open(f'{dir}/{dsname}/trainy.pickle','rb'))
    testX= pickle.load(open(f'{dir}/{dsname}/testX.pickle','rb'))
    testy= pickle.load(open(f'{dir}/{dsname}/testy.pickle','rb'))
    svtr = pickle.load(open(f'{dir}/{dsname}/svtr.pickle','rb'))
    svts = pickle.load(open(f'{dir}/{dsname}/svts.pickle','rb'))

    return model, trainX,trainy,testX,testy,svtr,svts

## Multivariate model training

In [9]:
import warnings
warnings.filterwarnings("ignore")



bg_size = 1000
window_len = 23
stride =10
shap_version = 'deep' #'window'
absshap=True #Wheather to use absolut values for shap, or use sign of shapp in changepoint detection
#limits due to memory limitations of GPUs
SEQUENCE_LENGTH_LIMIT=2000
DIMENSIONALITY_LIMIT=300
ACC_LIMIT=0

multivariate_ts_list = [
    "ArticularyWordRecognition", "AtrialFibrillation", "BasicMotions", "Cricket", "Epilepsy",
    "ERing", "EthanolConcentration", "FaceDetection", "FingerMovements", "HandMovementDirection",
    "Handwriting", "Heartbeat", "Libras", "LSST", "NATOPS", "PenDigits", "RacketSports",
    "SelfRegulationSCP1", "SelfRegulationSCP2", "UWaveGestureLibrary"
]





skipds=[]
summary = []
for p multivariate_ts_list:
    print(f'Time for {p}')
    if p in skipds:
        print(f'Skipping: {p}')
        continue
    try:
        X, y, meta_data = load_classification(p)
        X = np.moveaxis(X,1,2)
        if X.shape[1] > SEQUENCE_LENGTH_LIMIT:
            print(f'Skipping: {p} due to sequence length')
            summary.append([p,np.nan,'LENGTH'])
            continue
        if X.shape[2] > DIMENSIONALITY_LIMIT:
            print(f'Skipping: {p} due to dimensionality size')
            summary.append([p,np.nan,'DIMENSIONALITY'])
            continue
        
        print(f'Classifying: {p}')
        model, trainX, trainy, testX,testy,score=evaluate_lstmconv_model(p)
        if score < ACC_LIMIT:
            print(f'Skipping: {p} due to low score which is {score}')
            skipds.append(p)
            continue
        print(f'Size of the problem: {trainX.shape}')
        
        if shap_version == 'window':
            #There is a problem with Windowed version for more than two classes
            indexes = np.arange(0, len(trainX))
            np.random.shuffle(indexes)
            maxid = min(bg_size, len(trainX))
            background_data = trainX[indexes[:maxid]]
            
            sv_ts = np.zeros((len(testX),testX.shape[1], testX.shape[2]))
            sv_tr = np.zeros((len(trainX),trainX.shape[1], trainX.shape[2]))
            
            for i in range(len(testX)):
                gtw = SlidingWindowSHAP(model, stride, window_len, background_data, testX[i:i+1], model_type='lstm')
                sv_ts[i,:,:] = gtw.shap_values(num_output=trainy.shape[1])
            for i in range(len(trainX)):
                gtw = SlidingWindowSHAP(model, stride, window_len, background_data, trainX[i:i+1], model_type='lstm')
                sv_tr[i,:,:] = gtw.shap_values(num_output=trainy.shape[1])
        elif shap_version == 'deep':
            indexes = np.arange(0, len(trainX))
            np.random.shuffle(indexes)
            maxid = min(bg_size, len(trainX))
            background_data = trainX[indexes[:maxid]]
            
            explainer = shap.DeepExplainer(model,background_data)
            shap_values_ts = explainer.shap_values(testX, check_additivity=False)
            shap_values_tr = explainer.shap_values(trainX, check_additivity=False)
            if absshap:
                sv_ts = abs(np.array(shap_values_ts)).mean(axis=0) # This basically returns the average importance over the feature/sample
                                                   # Not taking into account the sign of shap value, as it is not required
                                                   # for breakpoints calculation
                sv_tr = abs(np.array(shap_values_tr)).mean(axis=0) # This basically returns the average importance over the feature/sample
                                                       # Not taking into account the sign of shap value, as it is not required
                                                       # for breakpoints calculation
            else:
                indexer = np.argmax(model.predict(testX), axis=1)
                sv_ts=[]
                for i in range(0,len(testX)):
                    sv_ts.append([shap_values_ts[indexer[i]][i,:]])
                sv_ts=np.concatenate(sv_ts)
                indexer = np.argmax(model.predict(trainX), axis=1)
                sv_tr=[]
                for i in range(0,len(trainX)):
                    sv_tr.append([shap_values_tr[indexer[i]][i,:]])
                sv_tr=np.concatenate(sv_tr)

        save_bundle(model, trainX,trainy,testX,testy,sv_tr,sv_ts, p,dir='./results/multivariate-models')
        summary.append([p,score,'OK'])
            
        tf.keras.backend.clear_session()
    except:
        summary.append([p,np.nan,'Exception'])
        print(f'Failed...')
        skipds.append(p)
        tf.keras.backend.clear_session()

multi_model_summary = './results/multivariate-summary.csv'
pd.DataFrame(summary,columns=['dataset','score','status']).to_csv(multi_model_summary,index=False)
    

## Univariate model training

In [10]:
import warnings
warnings.filterwarnings("ignore")


bg_size = 1000
window_len = 23
stride =10
shap_version = 'deep' #'window'
#limits due to memory limitations of GPUs
SEQUENCE_LENGTH_LIMIT=1000
DIMENSIONALITY_LIMIT=200
ACC_LIMIT=0
absshap=True

skipds = []
summary=[]

univariate_ts_list = [
    "Adiac", "Beef", "BeetleFly", "BirdChicken", "BME", "CBF", "Chinatown", "Coffee", "Computers",
    "CricketX", "CricketY", "CricketZ", "Crop", "DiatomSizeReduction", "DistalPhalanxOutlineAgeGroup",
    "DistalPhalanxOutlineCorrect", "DistalPhalanxTW", "DodgerLoopDay", "DodgerLoopGame", "DodgerLoopWeekend",
    "Earthquakes", "ECG200", "ECG5000", "ECGFiveDays", "ElectricDevices", "FaceFour", "FiftyWords", "FordA",
    "FordB", "FreezerRegularTrain", "FreezerSmallTrain", "Fungi", "GunPoint", "GunPointAgeSpan",
    "GunPointMaleVersusFemale", "GunPointOldVersusYoung", "Herring", "InsectWingbeatSound",
    "ItalyPowerDemand", "LargeKitchenAppliances", "Lightning2", "Lightning7", "Meat", "MedicalImages",
    "MiddlePhalanxOutlineAgeGroup", "MiddlePhalanxOutlineCorrect", "MiddlePhalanxTW", "MoteStrain", "OliveOil",
    "OSULeaf", "PhalangesOutlinesCorrect", "Plane", "PowerCons", "ProximalPhalanxOutlineAgeGroup",
    "ProximalPhalanxOutlineCorrect", "ProximalPhalanxTW", "RefrigerationDevices", "ScreenType", "ShapeletSim",
    "ShapesAll", "SmallKitchenAppliances", "SmoothSubspace", "SonyAIBORobotSurface1", "SonyAIBORobotSurface2",
    "Strawberry", "SwedishLeaf", "Symbols", "SyntheticControl", "ToeSegmentation2", "Trace", "TwoLeadECG",
    "TwoPatterns", "UMD", "UWaveGestureLibraryAll", "UWaveGestureLibraryX", "UWaveGestureLibraryY",
    "UWaveGestureLibraryZ", "Wafer", "Wine", "WordSynonyms", "Worms", "WormsTwoClass", "Yoga"
]



for p in univariate_ts_list:

    if p == continue_after:
        continue_after=None
        continue
    elif continue_after is not None:
        continue
    
    print(f'Time for {p}')
    if p in skipds:
        print(f'Skipping: {p}')
        continue
    try:
        X, y, meta_data = load_classification(p)
        X = np.moveaxis(X,1,2)
        if X.shape[1] > SEQUENCE_LENGTH_LIMIT:
            print(f'Skipping: {p} due to sequence length')
            summary.append([p,np.nan,'LENGTH'])
            continue
        if X.shape[2] > DIMENSIONALITY_LIMIT:
            print(f'Skipping: {p} due to dimensionality size')
            summary.append([p,np.nan,'DIMENSIONALITY'])
            continue

        print(f'Classifying: {p}')
        model, trainX, trainy, testX,testy,score=evaluate_lstmconv_model(p)
        if score < ACC_LIMIT:
            print(f'Skipping: {p} due to low score which is {score}')
            skipds.append(p)
            continue
        print(f'Size of the problem: {trainX.shape}')
        if shap_version == 'window':
            #There is a problem with Windowed version for more than two classes
            indexes = np.arange(0, len(trainX))
            np.random.shuffle(indexes)
            maxid = min(bg_size, len(trainX))
            background_data = trainX[indexes[:maxid]]
            
            sv_ts = np.zeros((len(testX),testX.shape[1], testX.shape[2]))
            sv_tr = np.zeros((len(trainX),trainX.shape[1], trainX.shape[2]))
            
            for i in range(len(testX)):
                gtw = SlidingWindowSHAP(model, stride, window_len, background_data, testX[i:i+1], model_type='lstm')
                sv_ts[i,:,:] = gtw.shap_values(num_output=trainy.shape[1])
            for i in range(len(trainX)):
                gtw = SlidingWindowSHAP(model, stride, window_len, background_data, trainX[i:i+1], model_type='lstm')
                sv_tr[i,:,:] = gtw.shap_values(num_output=trainy.shape[1])
        elif shap_version == 'deep':
            indexes = np.arange(0, len(trainX))
            np.random.shuffle(indexes)
            maxid = min(bg_size, len(trainX))
            background_data = trainX[indexes[:maxid]]
            
            explainer = shap.DeepExplainer(model,background_data)
            shap_values_ts = explainer.shap_values(testX, check_additivity=False)
            shap_values_tr = explainer.shap_values(trainX, check_additivity=False)
            if absshap:
                sv_ts = abs(np.array(shap_values_ts)).mean(axis=0) # This basically returns the average importance over the feature/sample
                                                   # Not taking into account the sign of shap value, as it is not required
                                                   # for breakpoints calculation
                sv_tr = abs(np.array(shap_values_tr)).mean(axis=0) # This basically returns the average importance over the feature/sample
                                                       # Not taking into account the sign of shap value, as it is not required
                                                       # for breakpoints calculation
            else:
                indexer = np.argmax(model.predict(testX), axis=1)
                sv_ts=[]
                for i in range(0,len(testX)):
                    sv_ts.append([shap_values_ts[indexer[i]][i,:]])
                sv_ts=np.concatenate(sv_ts)
                indexer = np.argmax(model.predict(trainX), axis=1)
                sv_tr=[]
                for i in range(0,len(trainX)):
                    sv_tr.append([shap_values_tr[indexer[i]][i,:]])
                sv_tr=np.concatenate(sv_tr)
                
        save_bundle(model, trainX,trainy,testX,testy,sv_tr,sv_ts, p,dir='./results/univariat-models')
        
        summary.append([p,score,'OK'])
            
        tf.keras.backend.clear_session()
    except:
        summary.append([p,np.nan,'Exception'])
        print(f'Failed...')
        skipds.append(p)
        tf.keras.backend.clear_session()

uni_model_summary = './results/univariate-summary.csv'
pd.DataFrame(summary,columns=['dataset','score','status']).to_csv(uni_model_summary,index=False)
   

## How to load pretrained model?

In [12]:
model, trainXo,trainyo,testX,testy,trainsvo,testsv = load_bundle('Crop',dir='./results/univariate-models')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2025-04-09 06:30:26.883190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 06:30:27.148368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12282 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:51:00.0, compute capability: 8.6
2025-04-09 06:30:27.260125: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2025-04-09 06:30:28.065957: W tensorflow/c/c_api.cc:291] Operation '{name:'dense/bias/Assign' id:665 op device:{requested: '', assigned: ''} def:{{{node dense/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 23, 2, 1)          0         
                                                                 
 conv_lstm1d (ConvLSTM1D)    (None, 23, 2, 64)         150016    
                                                                 
 conv_lstm1d_1 (ConvLSTM1D)  (None, 23, 2, 32)         110720    
                                                                 
 dropout (Dropout)           (None, 23, 2, 32)         0         
                                                                 
 embedding (Flatten)         (None, 1472)              0         
                                                                 
 dense (Dense)               (None, 100)               147300    
                                                                 
 dense_1 (Dense)             (None, 24)                2